# Testing Morphological Operators
Author: Nana Owusu

Purpose: To generate a mask of fMRI images of NIfTI format by
way of mathematical morphology.
This is an attempt at replicating the
first part of the RATS algorithm by Oguz et al.

RATS algorithm: https://www.iibi.uiowa.edu/rats-overview

methods: https://www.sciencedirect.com/science/article/abs/pii/S0165027013003440

In [ ]:
# Module for opening the image
# import matplotlib.image as mpimg
# from PIL import Image
# from pydicom import dcmread
from operator import itemgetter
from nibabel import load, Nifti1Image, save
from glob import glob
from math import floor

# Module for showing the image
import matplotlib.pyplot as plt

# Module for image morphology operators
import numpy as np 
# from skimage.measure import regionprops
from scipy.ndimage import binary_erosion, binary_dilation, grey_erosion, \
generate_binary_structure, binary_fill_holes, label, sum

In [ ]:
def multi_slice_display(input_vol):
    """Display all the slices of a single
    timepoint"""
    
    _, _, o = input_vol.shape
    
    # Instantiate a Figure object with one row
    # and multiple columns of axes.
    fig, axes = plt.subplots(1, o, figsize=[28, 35])
    fig.subplots_adjust(wspace=0.125)
    
    # Fill the figure with each slice
    for i,ax in enumerate(axes.flat):
        ax.imshow(np.rot90(input_vol[:,:,i]), cmap='gray')
        ax.set_axis_off()
        
    plt.show

In [ ]:
class struct_el(object):
    
    def __init__(self):
        self.shape = \
        {'square': self.square,
         'cube': self.cube,
         'circle': self.circle,
         'sphere': self.sphere,
         '4neighbor': self.neighbor4,
         '8neighbor': self.neighbor8
        }
        
    def neighbor8(self, d):
        struct = np.zeros((5, 5), dtype='uint8')
        struct[1:4,:] = 1
        struct[:,1:4] = 1
        if d == 0:
            return struct.astype(np.bool)
        elif d is not 0:                
            buffer = [struct.copy() for _ in range(d)]
            new_struct = np.stack(buffer, axis=0)
            return new_struct.astype(np.bool)
        
    def neighbor4(self, d):
        struct = np.zeros((3, 3), dtype='uint8')
        struct[:,1] = 1
        struct[1,:] = 1
        if d == 0:
            return struct.astype(np.bool)
        elif d is not 0:                
            buffer = [struct.copy() for _ in range(d)]
            new_struct = np.stack(buffer, axis=0)
            return new_struct.astype(np.bool)
        
    def square(self, w):
        struct = np.zeros((w, w), dtype='uint8')
        x, y = np.indices((w, w))
        mask = (x - w) * (y - w) <= n**2
        struct[mask] = 1
        return struct.astype(np.bool)

    def cube(self, w):
        struct = np.zeros((2 * w + 1, 2 * w + 1, 2 * w + 1))
        x, y, z = np.indices((2 * w + 1, 2 * w + 1, 2 * w + 1))
        mask = (x - w) *  (y - w) * (z - w) <= w**3
        struct[mask] = 1
        return struct.astype(np.bool)

    def circle(self, r):
        struct = np.zeros((2 * r + 1, 2 * r + 1))
        x, y = np.indices((2 * r + 1, 2 * r + 1))
        mask = (x - r)**2 + (y - r)**2 <= r**2
        struct[mask] = 1
        return struct.astype(np.bool)

    def sphere(self, r):
        struct = np.zeros((2 * r + 1, 2 * r + 1, 2 * r + 1))
        x, y, z = np.indices((2 * r + 1, 2 * r + 1, 2 * r + 1))
        mask = (x - r)**2 + (y - r)**2 + (z - r)**2 <= r**2
        struct[mask] = 1
        return struct.astype(np.bool)

In [ ]:
def find_largest_component(filled_binary):
    """Find the largest connected component in the binary image.
    This function labels the binary image and finds the summation
    of pixels in the labeled regions. The regions with the max pixels
    are chosen. Method adopted from https://tinyurl.com/y3m4qdg4
    
    filled_binary: Binary image without holes
    """
    # Label the regions in the binary image.
    labeled_im, num_of_regions =  label(filled_binary)
    
    # Sum up all pixels in the regions of the labeled binary.
    sizes = sum(filled_binary, labeled_im, range(num_of_regions + 1))
    
    # Store indicies of the regions with the greatest
    # number of pixels.
    mask = sizes == max(sizes)
    
    largest_regions = mask[labeled_im]
    
    return largest_regions, num_of_regions

In [ ]:
# file_loc = './fMRI_fMRI_(64x64)_FOV=2cm_TR=1s_20201109125634_4.nii'
file_loc = '/Users/nowusu/Documents/axelson_test/bold_air_fMRI_(96x96)_FOV=2cm_20210114183204_7.nii'
# nii_files = glob(file_loc + '*.dcm')
nii_obj = load(file_loc)
nii_hdr = nii_obj.header

im_data = nii_obj.get_fdata()

In [ ]:
# test_img = mpimg.imread('/Users/nowusu/Desktop/slackProfileIm.png')
# show_img = plt.imshow(test_img, cmap='gray', vmax=255, vmin=0)
# show_img = plt.imshow(im_data[:,:,0,0], cmap='gray')
# show_img.axes.set_axis_off()

# dcm_fig, dcm_ax = plt.subplots(1,1)

# def rgb2gray(rgb):
#     """Convert RGB image to grayscale image.
#     Image data of the former is dim=3 and the latter dim=1"""
#     return dot(rgb[:,:,:3], [0.2989, 0.5870, 0.1140])

# real_gray = rgb2gray(test_img)

# grayscale_img = Image.open('/Users/nowusu/Desktop/slackProfileIm.png')

# w, h = grayscale_img.size
# grayscale_img = grayscale_img.resize(size=(int(w/4), int(h/4)))
# grayscale_img.convert('LA')

# np_img = array(grayscale_img)

In [ ]:
multi_slice_display(im_data[:,:,:,0])

In [ ]:
sphere_stel = struct_el()
disk_H = sphere_stel.shape['sphere'](1)
footprints=generate_binary_structure(3, 2)

In [ ]:
eroded_im1 = grey_erosion(im_data[:,:,:,0], footprint=footprints, structure=disk_H)

In [ ]:
m, n, o, t = im_data.shape
threshold_im1 = np.ones((m, n, o), dtype='uint8')
threshold_im1[eroded_im1 < 750] = 0

### Hole-filling step

In [ ]:
square_stel = struct_el()
new_4neighbor = square_stel.shape['4neighbor'](1)
# new_4neighbor

In [ ]:
filled_holes = binary_fill_holes(threshold_im1, structure=new_4neighbor)
threshold1_complement = 1 - filled_holes

In [ ]:
multi_slice_display(threshold1_complement)

### Finding the largest connected component
code for this was adopted from ...
https://tinyurl.com/y62zqely

In [ ]:
largest_reg1, num_of_regions = find_largest_component(filled_holes)
num_of_regions

In [ ]:
multi_slice_display(largest_reg1)

In [ ]:
affine = nii_obj.affine
hdr = nii_obj.header
fmrIm = Nifti1Image(largest_reg1.astype(np.float32), affine)
fmrIm.header.set_qform(hdr.get_qform())

oldSform = hdr.get_sform(coded=True)
fmrIm.header.set_sform(oldSform[0],oldSform[1])

fmrIm.header.set_data_shape(hdr.get_data_shape())

save(fmrIm, filename='/Users/nowusu/Documents/axelson_test/fmr_mm_mask.nii.gz')

### Binary Opening Step

In [ ]:
### Binary erosion of a binary union ###
union_im = threshold_im1 | largest_reg1
disk_H2 = sphere_stel.shape['sphere'](1)
bin_erode = binary_erosion(union_im, structure=disk_H2)

largest_reg2, num_of_regions2 = find_largest_component(bin_erode)
num_of_regions2

In [ ]:
multi_slice_display(largest_reg2)

In [ ]:
### Binary dilation of a binary intersect ###
# im1 = np.where(threshold_im1 > 0, 1, 0)
im1 = np.where(threshold1_complement < 1, 1, 0)
im2 = np.where(bin_erode > 0, 1, 0)

intersect = im1 & im2
dilate_im2 = binary_dilation(intersect, disk_H)

In [ ]:
multi_slice_display(dilate_im2)